In [45]:
import pairTrade
from pairTrade import pairTrade
import pandas as pd
import warnings
import pandas.util.testing as tm
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.tsa.stattools as ts 
import statsmodels.formula.api as sm
warnings.filterwarnings('ignore')

## plot
%matplotlib inline
plt.figure(figsize = (20,20))

symbols = pd.read_csv(f"/Users/jochien/Documents/MaxRereach/backtrading/rawdata/1D/symbols.csv").symbol.tolist()
df = ((pd.concat([pd.read_csv(f"/Users/jochien/Documents/MaxRereach/backtrading/rawdata/1D/{symbol}.csv", index_col='date', parse_dates=True)
        ['close'].rename(symbol)for symbol in symbols],axis=1,sort=True)))
df = df.loc[:,~df.columns.duplicated()]

<Figure size 1440x1440 with 0 Axes>

# pair selection

In [47]:
### 波動度
def volatilityFiliter(df, tradeType, threshold = None, rolling = 20, plot = True):
    """
        Args:
            df (dataframe): raw data 
            tradeType (string): priceSpread, priceRatio, returnSpread, regression. 
            threshold (int):  
            rolling (int):  
    """
    tempList = []
    for A_symbol in df.columns:
        for B_symbol in df.columns:
            if A_symbol != B_symbol:
                if tradeType == 'priceSpread':
                    para = df[A_symbol] - df[B_symbol]
                elif tradeType == 'priceRatio':
                    para = df[A_symbol]/df[B_symbol]
                elif tradeType == 'returnSpread':
                    A_return = df[A_symbol].pct_change()
                    B_return = df[B_symbol].pct_change()
                    para = A_return - B_return
                elif tradeType == 'regression':
                    hedgeRatio=np.full(self.df.shape[0], 0.0)
                    for t in np.arange(self.rolling, len(hedgeRatio)):
                        regress_results=sm.ols(formula="{} ~ {}".format(A_Symbol, B_Symbol), data = df[(t-rolling):t]).fit() # Note this can deal with NaN in top row
                        hedgeRatio[t-1]=regress_results.params[1] ## beta1
                        para = df[A_symbol] - hedgeRatio * df[B_symbol]        
                zscore = (para-para.mean())/para.std()
                signal = ((zscore.shift(1)>0) & (zscore<=0)) | ((zscore.shift(1)<=0) & (zscore>0))
                num = len(signal[signal])
                tempList.append([A_symbol, B_symbol, num])
    result = pd.DataFrame(tempList, columns=['A_symbol', 'B_symbol', 'num']).sort_values('num').reset_index(drop=True)
    print('[average volatility] :', result['num'].mean())
    resultPivot = result.pivot("A_symbol", "B_symbol", "num")
    if plot:
        sns.heatmap(resultPivot, annot = True, xticklabels=1, yticklabels=1)
    if threshold is not None:
        resultFiliter = result[result['num'] >= threshold].reset_index(drop=True)
        return [resultFiliter['A_symbol'][i] + '_' + resultFiliter['B_symbol'][i] for i in range(resultFiliter.shape[0])]
        

## 相關性        
def correlation(df, threshold = None, plot = True):
    result = df.corr(method ='pearson')
    if plot:
        sns.heatmap(df.corr(), annot = True, xticklabels=1, yticklabels=1)

    if threshold is not None:
        resultFiliter = []
        for A_symbol in result.columns:
            for B_symbol in result.columns:
                if result.loc[A_symbol, B_symbol] >= threshold and A_symbol != B_symbol:
                    resultFiliter.append(A_symbol + '_' + B_symbol)
                    
    return resultFiliter

def minimum(df, threshold = None, plot = True):
    tempList = []
    for A_symbol in df.columns:
        for B_symbol in df.columns:
            if A_symbol != B_symbol:
                A_zscore = (df[A_symbol]-df[A_symbol].mean())/df[A_symbol].std()
                B_zscore = (df[B_symbol]-df[B_symbol].mean())/df[B_symbol].std()
                num = ((A_zscore-B_zscore)**2).sum()
                tempList.append([A_symbol, B_symbol, num])
    result = pd.DataFrame(tempList, columns=['A_symbol', 'B_symbol', 'num']).sort_values('num').reset_index(drop=True)
    print('[average minimum] :', result['num'].mean())
    resultPivot = result.pivot("A_symbol", "B_symbol", "num")
    if plot:
        sns.heatmap(resultPivot, annot = True, xticklabels=1, yticklabels=1)
    if threshold is not None:
        resultFiliter = result[result['num'] >= threshold].reset_index(drop=True)
        return [resultFiliter['A_symbol'][i] + '_' + resultFiliter['B_symbol'][i] for i in range(resultFiliter.shape[0])]

def cointegration(df, threshold = None,  plot = True):
    tempList = []
    for A_symbol in df.columns:
        for B_symbol in df.columns:
            if A_symbol != B_symbol:
                coin = ts.coint(df[A_symbol], df[B_symbol])
                tempList.append([A_symbol, B_symbol, coin[1]])
    result = pd.DataFrame(tempList, columns=['A_symbol', 'B_symbol', 'num']).sort_values('num').reset_index(drop=True)
    print('[average minimum] :', result['num'].mean())
    resultPivot = result.pivot("A_symbol", "B_symbol", "num")
    if plot:
        sns.heatmap(resultPivot, annot = True, xticklabels=1, yticklabels=1)
    if threshold is not None:
        resultFiliter = result[result['num'] < threshold].reset_index(drop=True)
        print(resultFiliter)
        return [resultFiliter['A_symbol'][i] + '_' + resultFiliter['B_symbol'][i] for i in range(resultFiliter.shape[0])]

In [ ]:
vol = volatilityFiliter(df, 'priceSpread', threshold = 40, plot = False)
corr = correlation(df, 0.6, plot = False)
mini = minimum(df, 0.6, plot = False)
coin = cointegration(df, 0.05, plot = False)


interSection = list(set(vol) & set(corr) & set(mini) & set(coin))

print("[vol]", len(vol))
print("[corr]", len(corr))
print("[mini]", len(mini))
print("[coin]", len(coin))
print("[interSection]", len(interSection))

[average volatility] : 13.196301564722617
[average minimum] : 289.9307122918183


In [44]:
df1 = df[['MATICUSDT', 'ATOMUSDT']]
A = pairTrade(df1)
A.indicator(tradeType = 'priceRatio')
A.strategy(strategyType = 'divergence', actionType = 'amount', entry = 1, exit = 0.2, stopLossType = 'trailingStop', trailingProfitTaking = 0.1, trailingStoploss = 0.05)
A.pplot()

[Stop-loss point] 2020-02-07 00:00:00
[Stop-loss point] 2020-02-23 00:00:00
[Stop-loss point] 2020-03-08 00:00:00
[Stop-loss point] 2020-03-19 00:00:00
[Stop-loss point] 2020-05-21 00:00:00
[Stop-loss point] 2020-06-12 00:00:00
[Stop-loss point] 2020-06-29 00:00:00
[Stop-loss point] 2020-07-13 00:00:00
[Stop-loss point] 2020-08-15 00:00:00
[Stop-loss point] 2020-09-25 00:00:00
[Stop-loss point] 2020-11-06 00:00:00
[Stop-loss point] 2020-11-26 00:00:00
[Stop-loss point] 2020-12-26 00:00:00
[Stop-loss point] 2021-01-01 00:00:00
[Stop-loss point] 2021-01-10 00:00:00
[Stop-loss point] 2021-01-28 00:00:00
[Stop-loss point] 2021-02-11 00:00:00
[Stop-loss point] 2021-02-23 00:00:00
[Stop-loss point] 2021-04-13 00:00:00
[Stop-loss point] 2021-04-30 00:00:00
[Stop-loss point] 2021-05-19 00:00:00
[Stop-loss point] 2021-07-13 00:00:00
[Stop-loss point] 2021-08-16 00:00:00
[Stop-loss point] 2021-09-04 00:00:00
[Stop-loss point] 2021-09-15 00:00:00
[Stop-loss point] 2021-10-19 00:00:00
[Stop-loss p

In [40]:
norm_prices = df.divide(df.iloc[0])

plt.figure(figsize = (15, 10))
plt.plot(norm_prices)
plt.xlabel('days')
plt.title('Performance of cryptocurrencies')
plt.legend(symbols)
plt.show()

result_list = []
for a1 in df.columns:
    for a2 in df.columns:
        if a1 != a2:
            test_result = ts.coint(df[a1], df[a2])
            if test_result[1] < 0.05:
                print(a1 + ' and ' + a2 + ': p-value = ' + str(test_result[1]))
                result_list.append((a1, a2, test_result[1]))

0.6594786814133669
0.4659018293204724
0.39285897360541533
0.114378083454026
0.4292484217850586
0.4265324854907845
0.32873267095641556
0.6701531432504505
0.572412219599535
0.17151963297855238
0.1425162521509813
0.8332093813053879
0.11624524795242697
0.2555033588664755
0.529034796509313
0.4762537908369432
0.48358903553709465
0.22970619558085126
0.24032725993912724
0.38735930449839295
0.3060084865777346
0.2042490383481116
0.9069768427755731
0.4555834816323408
0.28749515972386763
0.29141459102374623
0.2751762182073657
0.1241522884512854
0.08830642758763363
0.3477420086085487
0.4086430057511416
0.395327988099297
0.49367389201047884
0.28859363505410957
0.3175089961796459
0.5585583101483531
0.28082671745469323
0.6227111256294616
0.03979705332444582
ETHUSDT and BNBUSDT: p-value = 0.03979705332444582
0.7693838880317003
0.8322920955749031
0.78654705904321
0.286619489547692
0.12961471081171505
0.9015400462659414
0.07557160820903587
0.5232224654186005
0.9505792890519227
0.922375449598538
0.1377121

0.22763094474795786
0.4076558588227595
0.00988895178131794
XRPUSDT and FETUSDT: p-value = 0.00988895178131794
0.06188596109832435
0.13064203591790957
0.18115550638043065
0.02642596059284993
XRPUSDT and IOSTUSDT: p-value = 0.02642596059284993
0.028846465286064105
XRPUSDT and CELRUSDT: p-value = 0.028846465286064105
0.3137722866673309
0.04074986299658593
XRPUSDT and OMGUSDT: p-value = 0.04074986299658593
0.02785341862538615
XRPUSDT and THETAUSDT: p-value = 0.02785341862538615
0.09523656343527309
0.07491592922852142
0.1001879465484189
0.18433824001571464
0.25727433844049646
0.42803116596898844
0.6101791446494962
0.02300261222562835
EOSUSDT and NEOUSDT: p-value = 0.02300261222562835
0.26093432661193305
0.6175820497600224
0.249879078544478
0.33762973235378235
0.29034621677782957
0.13722584057600562
0.4024657025899452
0.2448505272318176
0.28176886474509166
0.5361936501112139
0.16844040274751942
0.061762618263361664
0.16750100382387112
0.05314334226619022
0.06561682549201575
0.434231007098011

KeyboardInterrupt: 

In [26]:
len(result_list)

322

* tradeType(priceRatio, priceSpread, returnSpread, priceRegression)
* strategyType: convergence 收斂策略, divergence 發散策略
* actionType: amount等金額購買, unit依比例購買
* entry 進場條件
* exit 出場條件
* signalStopLoss Defaults is None
* stopLoss 訊號停損
* singalStopLoss{固定停利停損 : fixedStopLoss, 移動停利停損 : trailingStop, 時間停利停損 : timeStop} 
* stopLossPara 停損參數 : fixedStopLoss 固定停利停損點位, trailingStop 移動停利停損點位, timeStop 時間停利停損時間間隔

# Ratio

### singal stoploss

In [ ]:
# for i in result_list[:1]:
# df1 = df[[i[0], i[1]]]
df1 = df[['BTCUSDT', 'ETHUSDT']]
A = pairTrade(df1)
A.indicator(tradeType = 'priceRatio')
A.strategy(strategyType = 'convergence', actionType = 'amount', entry = 1, exit = 0, signalStopLoss = None)
A.pplot()

### stopLoss (fixedProfitTaking)